In [1]:
import numpy as np
import pandas as pd

In [2]:
import multiprocessing
import os 
multiprocessing.set_start_method("spawn", force=True)
print("Number of processors: ", multiprocessing.cpu_count())
os.environ['JULIA_NUM_THREADS'] = str(multiprocessing.cpu_count())

Number of processors:  12


In [3]:
import juliacall
print('threads: ', juliacall.Main.Threads.nthreads())

threads:  12


In [4]:
juliacall.Main.seval('using Distributed')
jl = juliacall.Main
jl.addprocs(11)

11-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [5]:
# jl.rmprocs(jl.workers())
jl.procs()
jl.workers()

11-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [6]:
jl.seval('using BioSimulator')
model = jl.Network("multi")  

initial = 2

model <= jl.Species("SS",int(initial))
model <= jl.Species("L1",0)
model <= jl.Species("L2",0)
model <= jl.Species("R1",0)
model <= jl.Species("R2",0)
model <= jl.Species("D",0)

kf_dl = 4e8
nonspecific_correction = 1/5

sliding_correction = 1/2
kf_sliding = 1e7 * sliding_correction

Na = 6.023e23

kf_ssl1 = kf_dl*nonspecific_correction
kb_ssl1 = 22.920792678077465 

kf_ssl2 = kf_dl*nonspecific_correction
kb_ssl2 = 0.007808225837362128

kf_ssr2 = kf_dl*nonspecific_correction
kb_ssr2 = 0.0626785751957368

kf_ssr1 = kf_dl*nonspecific_correction
kb_ssr1 = 183.99091641848813

kf_ssD = kf_dl*nonspecific_correction
kb_ssD = 6.902831840829526e-08

kf_l1l2 = kf_sliding
kb_l1l2 = 3406.6124793451313*sliding_correction

kf_l2D = kf_sliding
kb_l2D = 88.40461309148736*sliding_correction

kf_r1r2 = kf_sliding
kb_r1r2 = 3406.6124793451313*sliding_correction

kf_r2D = kf_sliding
kb_r2D = 11.01306438328072*sliding_correction

model <= jl.Reaction("ss-l1_f",  kf_ssl1, "SS + SS --> L1")
model <= jl.Reaction("ss-l1_b",  kb_ssl1, "L1 --> SS + SS")
model <= jl.Reaction("ss-l2_f",  kf_ssl2, "SS + SS --> L2")
model <= jl.Reaction("ss-l2_b",  kb_ssl2, "L2 --> SS + SS")
model <= jl.Reaction("ss-r2_f",  kf_ssr2, "SS + SS --> R2")
model <= jl.Reaction("ss-r2_b",  kb_ssr2, "R2 --> SS + SS")
model <= jl.Reaction("ss-r1_f",  kf_ssr1, "SS + SS --> R1")
model <= jl.Reaction("ss-r1_b",  kb_ssr1, "R1 --> SS + SS")
model <= jl.Reaction("ss-D_f",   kf_ssD, "SS + SS --> D")
model <= jl.Reaction("ss-D_b",   kb_ssD, "D --> SS + SS")
model <= jl.Reaction("l1-l2_f",  kf_l1l2, "L1 --> L2")
model <= jl.Reaction("l1-l2_b",  kb_l1l2, "L2 --> L1")
model <= jl.Reaction("l2-D_f",   kf_l2D, "L2 --> D")
model <= jl.Reaction("l2-D_b",   kb_l2D, "D --> L2")
model <= jl.Reaction("r1-r2_f",  kf_r1r2, "R1 --> R2")
model <= jl.Reaction("r1-r2_b",  kb_r1r2, "R2 --> R1")
model <= jl.Reaction("r2-D_f",   kf_r2D, "R2 --> D")
model <= jl.Reaction("r2-D_b",   kb_r2D, "D --> R2")

Nmonte = 100000

# def run_model():
#     import juliacall
#     jl = juliacall.newmodule('multi')
#     jl.seval('using BioSimulator')
#     model = make_model()
#     runtime = 1e-5
#     return jl.simulate(model, jl.Direct(),tfinal=runtime)

# def run_sim():
#     model = make_model()
#     result = run_model()
#     return result

# if __name__ == '__main__':
#     with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
#         results = pool.map(run_sim(), range(Nmonte))

jl.seval("""
@everywhere begin
    function run_model()
        simulate(model, Direct(),tfinal=1e-5)
    end
end
""")

sim = jl.pmap(jl.run_model, range(Nmonte))

# res = jl.simulate(model, jl.Direct(),tfinal=1e-5)
# simresults = [jl.simulate(model, jl.Direct(),tfinal=1e-5) for i in range(Nmonte)]
# results = [{'c':jl.hcat(*simresults[i].u).__array__(), 't':simresults[i].t} for i in range(Nmonte)]
# simresults = juliacall.VectorValue
# for i in range(Nmonte):
#     jl.append(simresults,jl.simulate(model, jl.Direct(),tfinal=runtime))


JuliaError: On worker 3:
MethodError: no method matching run_model(::Int64)
Closest candidates are:
  run_model() at none:3 (method too new to be called from this world context.)
Stacktrace:
 [1] #invokelatest#2
   @ ./essentials.jl:729
 [2] invokelatest
   @ ./essentials.jl:726
 [3] #110
   @ ~/TESI-CODE/hDNA/virtualenvs/julia-hdna/pyjuliapkg/install/share/julia/stdlib/v1.8/Distributed/src/process_messages.jl:285
 [4] run_work_thunk
   @ ~/TESI-CODE/hDNA/virtualenvs/julia-hdna/pyjuliapkg/install/share/julia/stdlib/v1.8/Distributed/src/process_messages.jl:70
 [5] macro expansion
   @ ~/TESI-CODE/hDNA/virtualenvs/julia-hdna/pyjuliapkg/install/share/julia/stdlib/v1.8/Distributed/src/process_messages.jl:285 [inlined]
 [6] #109
   @ ./task.jl:484
Stacktrace:
  [1] (::Base.var"#939#941")(x::Task)
    @ Base ./asyncmap.jl:177
  [2] foreach(f::Base.var"#939#941", itr::Vector{Any})
    @ Base ./abstractarray.jl:2774
  [3] maptwice(wrapped_f::Function, chnl::Channel{Any}, worker_tasks::Vector{A

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#definitive routine for mfpis is detailed below

index = 2

results[index]['c']
results[index]['c'][-1]
results[index]['t']

fpi = np.argmax(results[index]['c'][-1] == 1)

fpi, results[index]['t'][fpi]

array([[2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 1, 1]])

array([0, 0, 0, 1, 0, 1, 0, 1, 1])

9-element Vector{Float64}:
 0.0
 2.372237859220639e-9
 1.4628208175078858e-7
 3.001033672771653e-7
 0.005699058104488336
 0.005699231462495758
 0.009411363406971164
 0.009411431725315063
 0.01

(3, 3.001033672771653e-07)

In [ ]:
# here i will keep the fastest routine i found for conversion
for sim in range(50000):
    # a = pd.DataFrame(simresults[sim].u)
    a = simresults[sim].u.__array__()


In [ ]:
@parfor(range(5000))
def fun(i):
    i = pd.concat([i, pd.DataFrame(simresults[sim].u)])

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
a

,0,1,2,3,4,5
0,2,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1
0,2,0,0,0,0,0
1,0,0,0,0,1,0
2,0,0,0,0,0,1
3,0,0,0,0,0,1
0,2,0,0,0,0,0


In [ ]:
pd.DataFrame(simresults[0].u)

,0,1,2,3,4,5
0,2,0,0,0,0,0
1,0,0,0,0,1,0
2,0,0,0,0,0,1
3,0,0,0,0,0,1


In [ ]:
type(simresults[1].to_numpy())
#NO NEED TO CONVERT IT TO PYTHON JUST CALL IT AS BELOW 
simresults[1].u[-1]

numpy.ndarray

6-element Vector{Int64}:
 0
 0
 0
 0
 0
 1

In [ ]:
def first(array, condition = lambda x: x == 1):
    for i in range(len(array)):
        if condition(array[i]):
            return i

In [ ]:
pd.DataFrame(simresults[0].u)
x=first(simresults[0].u[-1])
simresults[0].t

,0,1,2,3,4,5
0,2,0,0,0,0,0
1,0,0,0,0,1,0
2,0,0,0,0,0,1
3,0,0,0,0,0,1


4-element Vector{Float64}:
 0.0
 5.3295782525917545e-9
 2.2212434133477408e-7
 0.01

In [ ]:
index = 5
type(simresults[index])
len(simresults[index])
np.shape(simresults[index].__array__())
np.shape(simresults[index])

juliacall.ArrayValue

3

(6, 3)

(6, 3)